In [18]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import time

In [19]:
base = pd.read_csv('iris.csv')
base.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [20]:
previsores = base.iloc[:, 0:4].values
previsores[0:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [21]:
classe = base.iloc[:, 4].values
classe[0:3]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa'], dtype=object)

In [22]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)
classe[0:3]

array([0, 0, 0])

In [23]:
# Nao utilizado nos cross validation, uma vez que o sklearn ja faz esse processamento
from keras.utils import np_utils
classe_dummy = np_utils.to_categorical(classe)
classe_dummy[0:3]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [38]:
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback

In [39]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [56]:
def criarRede(unit1, unit2, kernel_initializer1, kernel_initializer2, drop1, drop2, optimizer):
    # cleanup
    k.clear_session()
    
    classificador = Sequential()
    
    classificador.add(Dense(units = unit1, activation = 'relu', 
                        kernel_initializer = kernel_initializer1, input_dim = 4))
    
    classificador.add(Dropout(drop1))
    
    classificador.add(Dense(units = unit2, activation = 'relu', 
                        kernel_initializer = kernel_initializer2))
    
    classificador.add(Dropout(drop2))
    
    classificador.add(Dense(units = 3, activation = 'softmax'))
    
    #otimizador = keras.optimizers.Adam(lr = 0.001, decay = 0.0001, clipvalue = clipvalue)
    
    classificador.compile(optimizer = optimizer, 
                          loss = 'sparse_categorical_crossentropy',
                          metrics = ['accuracy'])
                          #run_eagerly=True)
    
    return classificador

In [57]:
classificador = KerasClassifier(build_fn = criarRede)
parametros = {'batch_size': [10],
              'epochs': [300],
              'callbacks': [ClearMemory()],
              'drop1': [0.2, 0.3],
              'drop2': [0.2, 0.3],
              'unit1': [8],
              'unit2': [8],
              'kernel_initializer1': ['normal'],
              'kernel_initializer2': ['random_uniform'],
              'optimizer': ['RMSprop']
              #'clipvalue': [0.5, 0.1, 0.05]
             }

In [58]:
grid_search = GridSearchCV(estimator = classificador,
                           param_grid = parametros,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs=1)

In [59]:
from sklearn.utils import parallel_backend

In [60]:
start = time.time()
with parallel_backend('threading'):
    grid_search = grid_search.fit(previsores, classe)

Epoch 1/300
12/12 [==============================] - 0s 12ms/step - loss: 1.0938 - accuracy: 0.5089
Epoch 2/300
12/12 [==============================] - 0s 10ms/step - loss: 1.0835 - accuracy: 0.5089
Epoch 3/300
12/12 [==============================] - 0s 10ms/step - loss: 1.0779 - accuracy: 0.5714
Epoch 4/300
12/12 [==============================] - 0s 9ms/step - loss: 1.0680 - accuracy: 0.4643
Epoch 5/300
12/12 [==============================] - 0s 10ms/step - loss: 1.0550 - accuracy: 0.5982
Epoch 6/300
12/12 [==============================] - 0s 10ms/step - loss: 1.0458 - accuracy: 0.4911
Epoch 7/300
12/12 [==============================] - 0s 10ms/step - loss: 1.0298 - accuracy: 0.5089
Epoch 8/300
12/12 [==============================] - 0s 10ms/step - loss: 1.0099 - accuracy: 0.5625
Epoch 9/300
12/12 [==============================] - 0s 10ms/step - loss: 1.0115 - accuracy: 0.4911
Epoch 10/300
12/12 [==============================] - 0s 10ms/step - loss: 0.9815 - accuracy: 0.4643


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x0000019E393D4E20>, as the constructor either does not set or modifies parameter callbacks

In [61]:
end = time.time()
total = (end - start)/60
print('{:.2f}min'.format(total))

12.22min


In [62]:
melhores_parametros = grid_search.best_params_
melhores_parametros

{'batch_size': 10,
 'callbacks': <__main__.ClearMemory at 0x19e3a7a3ca0>,
 'drop1': 0.2,
 'drop2': 0.3,
 'epochs': 300,
 'kernel_initializer1': 'normal',
 'kernel_initializer2': 'random_uniform',
 'optimizer': 'RMSprop',
 'unit1': 8,
 'unit2': 8}

In [63]:
melhor_precisao = grid_search.best_score_
melhor_precisao

0.9262091038406829

In [54]:
melhores_parametros = grid_search.best_params_
melhores_parametros

{'batch_size': 10,
 'callbacks': <__main__.ClearMemory at 0x19e3770adf0>,
 'drop1': 0.2,
 'drop2': 0.2,
 'epochs': 300,
 'kernel_initializer1': 'normal',
 'kernel_initializer2': 'random_uniform',
 'optimizer': 'RMSprop',
 'unit1': 8,
 'unit2': 8}

In [55]:
melhor_precisao = grid_search.best_score_
melhor_precisao

0.9194523470839261